In [ ]:
pwd

'/content'

In [ ]:
cd drive/MyDrive/Colab Notebooks/TFC-pretraining-main/code/TFC

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/TFC-pretraining-main/code/TFC'
/content/drive/MyDrive/Colab Notebooks/TFC-pretraining-main


In [ ]:
ls

augmentations.py  Data_split.py  loss.py  model.py      readme      utils.py
dataloader.py     KNN.py         main.py  __pycache__/  trainer.py


In [ ]:
"""Updated implementation for TF-C -- Xiang Zhang, Jan 16, 2023"""

import os
import torch
import numpy as np
from datetime import datetime
import argparse
from utils import _logger
from model import *
from dataloader import data_generator
from trainer import Trainer




# Args selections
start_time = datetime.now()
parser = argparse.ArgumentParser()
######################## Model parameters ########################
home_dir = os.getcwd()
parser.add_argument('--run_description', default='run1', type=str,
                    help='Experiment Description')
parser.add_argument('--seed', default=42, type=int, help='seed value')

# 1. self_supervised pre_train; 2. finetune (itself contains finetune and test)
parser.add_argument('--training_mode', default='fine_tune_test', type=str,
                    help='pre_train, fine_tune_test')

parser.add_argument('--pretrain_dataset', default='SleepEEG', type=str,
                    help='Dataset of choice: SleepEEG, FD_A, HAR, ECG')
parser.add_argument('--target_dataset', default='Epilepsy', type=str,
                    help='Dataset of choice: Epilepsy, FD_B, Gesture, EMG')

parser.add_argument('--logs_save_dir', default='../experiments_logs', type=str,
                    help='saving directory')
parser.add_argument('--device', default='cuda', type=str,
                    help='cpu or cuda')
parser.add_argument('--home_path', default=home_dir, type=str,
                    help='Project home directory')
args, unknown = parser.parse_known_args()

with_gpu = torch.cuda.is_available()
if with_gpu:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print('We are using %s now.' %device)

pretrain_dataset = args.pretrain_dataset
targetdata = args.target_dataset
experiment_description = str(pretrain_dataset) + '_2_' + str(targetdata)


method = 'TF-C'
training_mode = args.training_mode
run_description = args.run_description
logs_save_dir = args.logs_save_dir
os.makedirs(logs_save_dir, exist_ok=True)
exec(f'from config_files.{pretrain_dataset}_Configs import Config as Configs')
configs = Configs()

# # ##### fix random seeds for reproducibility ########
SEED = args.seed
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
#####################################################

experiment_log_dir = os.path.join(logs_save_dir, experiment_description, run_description, training_mode + f"_seed_{SEED}_2layertransformer")
# 'experiments_logs/Exp1/run1/train_linear_seed_0'
os.makedirs(experiment_log_dir, exist_ok=True)

# loop through domains
counter = 0
src_counter = 0


# Logging
log_file_name = os.path.join(experiment_log_dir, f"logs_{datetime.now().strftime('%d_%m_%Y_%H_%M_%S')}.log")
# 'experiments_logs/Exp1/run1/train_linear_seed_0/logs_14_04_2022_15_13_12.log'
logger = _logger(log_file_name)
logger.debug("=" * 45)
logger.debug(f'Pre-training Dataset: {pretrain_dataset}')
logger.debug(f'Target (fine-tuning) Dataset: {targetdata}')
logger.debug(f'Method:  {method}')
logger.debug(f'Mode:    {training_mode}')
logger.debug("=" * 45)

# Load datasets
sourcedata_path = f"../../datasets/{pretrain_dataset}"
targetdata_path = f"../../datasets/{targetdata}"
subset = True  # if subset= true, use a subset for debugging.
train_dl, valid_dl, test_dl = data_generator(sourcedata_path, targetdata_path, configs, training_mode, subset = subset)
logger.debug("Data loaded ...")

# Load Model
"""Here are two models, one basemodel, another is temporal contrastive model"""
TFC_model = TFC(configs).to(device)
classifier = target_classifier(configs).to(device)
temporal_contr_model = None


if training_mode == "fine_tune_test":
    # load saved model of this experiment
    load_from = os.path.join(os.path.join(logs_save_dir, experiment_description, run_description,
    f"pre_train_seed_{SEED}_2layertransformer", "saved_models"))
    print("The loading file path", load_from)
    chkpoint = torch.load(os.path.join(load_from, "ckp_last.pt"), map_location=device)
    pretrained_dict = chkpoint["model_state_dict"]
    TFC_model.load_state_dict(pretrained_dict)

model_optimizer = torch.optim.Adam(TFC_model.parameters(), lr=configs.lr, betas=(configs.beta1, configs.beta2), weight_decay=3e-4)
classifier_optimizer = torch.optim.Adam(classifier.parameters(), lr=configs.lr, betas=(configs.beta1, configs.beta2), weight_decay=3e-4)

# Trainer
Trainer(TFC_model, model_optimizer, classifier, classifier_optimizer, train_dl, valid_dl, test_dl, device,
        logger, configs, experiment_log_dir, training_mode)

logger.debug(f"Training time is : {datetime.now()-start_time}")

We are using cuda now.


DEBUG:../experiments_logs/SleepEEG_2_Epilepsy/run1/fine_tune_test_seed_42_2layertransformer/logs_02_02_2023_06_17_45.log:=============================================


Pre-training Dataset: SleepEEG


DEBUG:../experiments_logs/SleepEEG_2_Epilepsy/run1/fine_tune_test_seed_42_2layertransformer/logs_02_02_2023_06_17_45.log:Pre-training Dataset: SleepEEG


Target (fine-tuning) Dataset: Epilepsy


...

DEBUG:../experiments_logs/SleepEEG_2_Epilepsy/run1/fine_tune_test_seed_42_2layertransformer/logs_02_02_2023_06_17_45.log:
################## Best testing performance! #########################


Best Testing Performance: Acc=94.6410| Precision = 93.0236 | Recall = 89.6766 | F1 = 91.2238 | AUROC= 96.1677 | AUPRC=94.0092
Best KNN F1 0.7610561290274263

################## Training is Done! #########################


DEBUG:../experiments_logs/SleepEEG_2_Epilepsy/run1/fine_tune_test_seed_42_2layertransformer/logs_02_02_2023_06_17_45.log:
################## Training is Done! #########################


Training time is : 0:01:13.809615


DEBUG:../experiments_logs/SleepEEG_2_Epilepsy/run1/fine_tune_test_seed_42_2layertransformer/logs_02_02_2023_06_17_45.log:Training time is : 0:01:13.809615


In [ ]:
cd TFC

/content/drive/MyDrive/Colab Notebooks/TFC-pretraining-main/code/TFC


In [ ]:
ls

augmentations.py  experiments_logs/  main.py       readme
dataloader.py     KNN.py             model.py      trainer.py
Data_split.py     loss.py            __pycache__/  utils.py


In [ ]:
!python main.py --training_mode pre_train --pretrain_dataset SleepEEG --target_dataset Epilepsy

We are using cuda now.
Pre-training Dataset: SleepEEG
Target (fine-tuning) Dataset: Epilepsy
Method:  TF-C
Mode:    pre_train
Using subset for debugging, the datasize is: 1280
Using subset for debugging, the datasize is: 60
Data loaded ...
Training started ....
Pretraining on source dataset
Pretraining: overall loss:8.839262962341309, l_t: 3.7407238483428955, l_f:6.083005428314209, l_c:6.874517440795898
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.8/logging/__init__.py", line 1085, in emit
    msg = self.format(record)
  File "/usr/lib/python3.8/logging/__init__.py", line 929, in format
    return fmt.format(record)
  File "/usr/lib/python3.8/logging/__init__.py", line 668, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.8/logging/__init__.py", line 373, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "main.py", line 116, in <module>
    Trainer(TF

In [ ]:
!python main.py --training_mode fine_tune_test --pretrain_dataset SleepEEG --target_dataset Epilepsy

We are using cuda now.
Pre-training Dataset: SleepEEG
Target (fine-tuning) Dataset: Epilepsy
Method:  TF-C
Mode:    fine_tune_test
Using subset for debugging, the datasize is: 1280
Using subset for debugging, the datasize is: 60
Data loaded ...
The loading file path ../experiments_logs/SleepEEG_2_Epilepsy/run1/pre_train_seed_42_2layertransformer/saved_models
Training started ....
Fine-tune on Fine-tuning set

Epoch : 1
 Finetune: loss = 7.4396| Acc=48.3333 | Precision = 41.2281 | Recall = 48.3333 | F1 = 35.2142| AUROC=36.7222 | AUPRC = 44.1919
update fine-tuned model
Test on Target datasts test set
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
MLP Testing: Acc=19.7898| Precision = 9.8949 | Recall = 50.0000 | F1 = 16.5205 | AUROC= 45.7111 